In [1]:
# Impordid
import os
import csv
from datasets import load_dataset
from src.transformers import BertTokenizer, BertForMaskedLM, BertConfig, Trainer, TrainingArguments, DataCollatorForLanguageModeling

In [2]:
test_andmed = load_dataset("json", data_files={'test': ["korpus/eval.json"]})["test"]

  0%|          | 0/1 [00:00<?, ?it/s]

In [3]:
from sklearn.metrics import accuracy_score
import numpy as np

def compute_metrics(p):
    pred, labels = p
    
    indeksid = np.where(labels[:, :, 0].flatten() != -100)[0]
    
    labels_lemma = labels[:, :, 0].flatten()[indeksid]
    labels_vorm = labels[:, :, 1].flatten()[indeksid]

    pred_lemma = np.take(np.argmax(pred[0], axis = 2).flatten(), indeksid)
    pred_vorm = np.take(np.argmax(pred[1], axis = 2).flatten(), indeksid)

    accuracy_lemma = accuracy_score(y_true=labels_lemma, y_pred=pred_lemma)
    accuracy_vorm = accuracy_score(y_true=labels_vorm, y_pred=pred_vorm)

    return {"accuracy_lemma": accuracy_lemma, "accuracy_vorm": accuracy_vorm}

tokenizer = BertTokenizer(vocab_file = "vocab_final.txt", vocab_file_form = "vocab_form.txt", max_length = 128, padding = "max_length", truncation = True, return_tensors = "pt")


data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer,
        mlm=True,
        mlm_probability=0.15)

model = BertForMaskedLM.from_pretrained("checkpoint-300000")

trainer = Trainer(
    model=model,
    data_collator = data_collator,
    eval_dataset = test_andmed,
    compute_metrics = compute_metrics
)

metrics = trainer.evaluate()

***** Running Evaluation *****
  Num examples = 238992
  Batch size = 8


KeyboardInterrupt: 

In [ ]:
metrics

In [ ]:
import torch
lemma_embedding = {lemma: model.get_input_embeddings()[0](torch.tensor(id))  for lemma, id in tokenizer.get_vocab().items()}

In [ ]:
lemma_embeddings =[model.get_input_embeddings()[0](torch.tensor(id)).tolist() for lemma, id in tokenizer.get_vocab().items()]

In [ ]:
from sklearn.decomposition import PCA
pca = PCA()
pca.fit(lemma_embeddings)


In [ ]:
import numpy as np
kumu = np.cumsum(pca.explained_variance_ratio_) > 0.9
np.argmax(kumu == True)

In [ ]:
form_embeddings =[model.get_input_embeddings()[1](torch.tensor(id)).tolist() for lemma, id in tokenizer.get_vocab_form().items()]
pca2 = PCA()
pca2.fit(form_embeddings)

In [ ]:
kumu2 = np.cumsum(pca2.explained_variance_ratio_) > 0.9
np.argmax(kumu2 == True)

In [ ]:
from transformers import BertModel, BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert = BertModel.from_pretrained('bert-base-uncased')


In [ ]:
bert_embeddings =[bert.get_input_embeddings()(torch.tensor(id)).tolist() for lemma, id in tokenizer.get_vocab().items()]

In [ ]:
pca3 = PCA()
pca3.fit(bert_embeddings)

In [ ]:
kumu3 = np.cumsum(pca3.explained_variance_ratio_) > 0.9
np.argmax(kumu3 == True)